In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
url='https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'
names=['id', 'clump_thickness', 'uniform_cell_size','uniform_cell_shape','marginal_adhesion'
       ,'single_epithelial_size','bare_nuclel','bland_chromatin','normal_nucleoli','mitoses','class']
df=pd.read_csv(url, names=names)